<a href="https://colab.research.google.com/github/kmayutrisna/SA_IMDB/blob/main/Amazon_Glove_CNN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary library
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense

In [2]:
#Get the dataset from gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Display top of data
train_data =pd.read_csv("drive/My Drive/data/train_amazon.csv")
test_data = pd.read_csv("drive/My Drive/data/test_amazon.csv")

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
# Text preprocessing
from nltk.corpus import stopwords
def preprocess_text(text):

    # Text cleaning
    text = re.sub('[^a-zA-Z]', ' ', text)

    #Single character removal
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)

    #Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)

    # Lowercasing
    text = text.lower()

    # Lowercasing
    text = text.split()

    # Stopword removal
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if not word in stop_words]

    # Join the tokens back to a single string
    text = ' '.join(text)
    return text

In [7]:
# Preprocess the text column
import re
train_data['text'] = train_data['text'].apply(preprocess_text)
test_data['text'] = test_data['text'].apply(preprocess_text)

In [8]:
X_train = train_data['text']
y_train = train_data['label']

X_test = test_data['text']
y_test = test_data['label']

In [9]:
# Tokenization and Padding
max_words = 10000
max_seq_length = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_seq_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_seq_length)

In [10]:
# Load GloVe Embeddings
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))

glove_path = 'drive/My Drive/data/glove.6B.100d.txt'.format(embedding_dim)

with open(glove_path, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        if word in tokenizer.word_index and tokenizer.word_index[word] < max_words:
            index = tokenizer.word_index[word]
            embedding_matrix[index] = np.array(values[1:], dtype='float32')

In [11]:
# Build CNN Model
model = keras.Sequential([
    Embedding(max_words, embedding_dim, input_length=max_seq_length, weights=[embedding_matrix], trainable=False),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(5),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
# Train the Model
epochs = 5
batch_size = 64

model.fit(X_train_padded, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/5
450/450 [==============================] - 30s 64ms/step - loss: 0.4742 - accuracy: 0.7653 - val_loss: 0.3592 - val_accuracy: 0.8424
Epoch 2/5
450/450 [==============================] - 25s 55ms/step - loss: 0.3378 - accuracy: 0.8544 - val_loss: 0.3858 - val_accuracy: 0.8269
Epoch 3/5
450/450 [==============================] - 27s 59ms/step - loss: 0.2558 - accuracy: 0.8957 - val_loss: 0.3913 - val_accuracy: 0.8383
Epoch 4/5
450/450 [==============================] - 23s 51ms/step - loss: 0.1719 - accuracy: 0.9352 - val_loss: 0.4295 - val_accuracy: 0.8375
Epoch 5/5
450/450 [==============================] - 26s 57ms/step - loss: 0.0996 - accuracy: 0.9654 - val_loss: 0.5184 - val_accuracy: 0.8308


In [14]:
# Evaluate the Model
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)


125/125 [==============================] - 5s 35ms/step


In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")
print(f"F1 Score: {f1*100:.2f}%")

Accuracy: 81.62%
Precision: 83.38%
Recall: 80.09%
F1 Score: 81.70%
